In [1]:
from power import *
from power.systems import System3Bus
from power.systems import IEEE14
from power.systems.b6l8 import B6L8
from power.systems.ieee118 import IEEE118
import pulp as pl
import numpy as np

In [2]:
b3 = System3Bus()
b14 = IEEE14()
b6l8 = B6L8()
ieee118 = IEEE118()

In [3]:
print(ieee118.generators)

[Generator(id=1, bus=1, p=0.000, q=0.000, p_range=[0.000,1.300], q_range=[-0.05,0.15]),, Generator(id=4, bus=4, p=-0.090, q=0.000, p_range=[0.000,1.000], q_range=[-3.0,3.0]),, Generator(id=6, bus=6, p=0.000, q=0.000, p_range=[0.000,1.500], q_range=[-0.13,0.5]),, Generator(id=8, bus=8, p=-0.280, q=0.000, p_range=[0.000,4.700], q_range=[-3.0,3.0]),, Generator(id=10, bus=10, p=4.500, q=0.000, p_range=[0.000,1.000], q_range=[-1.47,2.0]),, Generator(id=12, bus=12, p=0.850, q=0.000, p_range=[0.000,1.000], q_range=[-0.35,1.2]),, Generator(id=15, bus=15, p=0.000, q=0.000, p_range=[0.000,4.500], q_range=[-0.1,0.3]),, Generator(id=18, bus=18, p=0.000, q=0.000, p_range=[0.000,1.000], q_range=[-0.16,0.5]),, Generator(id=19, bus=19, p=0.000, q=0.000, p_range=[0.000,1.000], q_range=[-0.08,0.24]),, Generator(id=24, bus=24, p=-0.130, q=0.000, p_range=[0.000,2.500], q_range=[-3.0,3.0]),, Generator(id=25, bus=25, p=2.200, q=0.000, p_range=[0.000,1.000], q_range=[-0.47,1.4]),, Generator(id=26, bus=26, p=

In [18]:
class LinearDispatch2:
    def __init__(self, net: Network):
        """
        Inicializa e constrói o problema de despacho econômico linear para uma dada rede.
        """
        self.net = net
        self.problem = None

        # Initializing losses on each bus:
        for b in self.net.buses:
            b.loss = 0
        
    def _create_problem(self):
        """Cria o objeto do problema PuLP."""
        self.problem = pl.LpProblem("Linear_Economic_Dispatch_OO", pl.LpMinimize)
    
    def _create_variables(self):
        """Cria as variáveis de decisão e as anexa aos objetos da rede."""
        # Geração Ativa
        for g in self.net.generators:
            g.p_var = pl.LpVariable(f"Gen_{g.name}", lowBound=g.p_min, upBound=g.p_max)

        # Ângulo das Barras
        for b in self.net.buses:
            if b.bus_type == "Slack":
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=0, upBound=0)
            else:
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=-np.pi, upBound=np.pi)
                b.theta_var.setInitialValue(0)

    def _create_objective(self):
        """Define a função objetivo do problema (minimizar custo total)."""
        self.problem += pl.lpSum([g.cost_b * g.p_var for g in self.net.generators]), "Total_Generation_Cost"

    def _create_constraints(self):
        """Cria todas as restrições do modelo."""
        # 1. Definição de Fluxo (liga as variáveis de fluxo aos ângulos)
        for line in self.net.lines:
            if line.reactance == 0:
                raise ValueError(f"Line '{line.id}' has zero reactance.")
            
            flow_eq = (line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance
            self.problem += flow_eq <= line.flow_max_pu, f"Flow_limit_Max_{line.name}"
            self.problem += flow_eq >= -line.flow_max_pu, f"Flow_limit_Inverted_{line.name}"

        for b in self.net.buses:
            generation = pl.lpSum([g.p_var for g in b.generators])
            load = sum([l.p for l in b.loads]) + b.loss
            flow_in = 0
            flow_out = 0
            for l in self.net.lines:
                if l.from_bus == b: #The line starts at bus 'b', so it's an outgoing flow
                    flow_out += (b.theta_var - l.to_bus.theta_var) / l.reactance
            
                elif l.to_bus == b: #The line ends at bus 'b', so it's an incoming flow
                    flow_in += (l.from_bus.theta_var - b.theta_var) / l.reactance

            self.problem += generation + flow_in - flow_out == load, f"{b.name}_Power_Balance"
    
    def _build(self):
        """" Monta o Problema de Otimização"""
        print("Building Optimization Problem...")
        self._create_problem()
        self._create_variables()
        self._create_constraints()
        self._create_objective()

    def solve(self):
        """Resolve o problema de otimização."""
        self._build()
        print("Solving...")
        self.problem.solve()
        return self.problem.status == pl.LpStatusOptimal

    
    def solve_with_loss(self, iter_max=10, tol=1e-10):
        if not self.solve():
            raise ValueError("Não foi possível resolver o despacho inicial sem perdas")
        
        prev_total_loss = 0
        for i in range(iter_max):
            print(f"\n--- Iniciando Iteração {i+1} de Cálculo de Perdas ---")

            for b in self.net.buses:
                b.loss = 0
            
            current_total_loss = 0
            for l in self.net.lines:
                r = l.resistance
                x = l.reactance
                g_series = r / (r**2 + x**2) if (r**2 + x**2) > 0 else 0
                dtheta = l.from_bus.theta_var.value() - l.to_bus.theta_var.value()
                line_loss = g_series * (dtheta ** 2)
                current_total_loss += line_loss
                l.from_bus.loss += line_loss / 2
                l.to_bus.loss += line_loss / 2
            
            #Verificação de Convergência
            loss_diff = abs(current_total_loss - prev_total_loss)
            print(f"Mudança nas perdas desde a última iteração: {loss_diff:.6f} pu")
            if loss_diff < tol:
                print(f"Convergência atingida. A mudança nas perdas está abaixo da tolerância")
                break
            prev_total_loss = current_total_loss

            if not self.solve():
                print(f"Aviso: Não foi possível resolver o despacho com perdas. Erro nas iteração {i}")
            
        print("\n Processo de despacho com perdas finalizado")
        
    def print_model(self):
        print(self.problem)

    def print_results(self):
            """
            Imprime um relatório completo e organizado dos resultados da última solução,
            seguindo um formato tabular e estruturado.
            """
            if self.problem is None or self.problem.status != pl.LpStatusOptimal:
                status = "Não resolvido" if self.problem is None else pl.LpStatus[self.problem.status]
                print(f"Não foi possível encontrar uma solução ótima. Status: {status}")
                return

            print("\n" + "="*90)
            print("RELATÓRIO DO DESPACHO ECONÔMICO".center(90))
            print("="*90)

            # --- 1. RESUMO GERAL DO SISTEMA ---
            total_gen = sum(g.p_var.value() for g in self.net.generators)
            total_load_demand = sum(l.p for l in self.net.loads)
            total_loss = sum(b.loss for b in self.net.buses)
            total_cost = pl.value(self.problem.objective)

            print("\n## 1. RESUMO GERAL DO SISTEMA ##")
            print(f"   - Custo Total de Operação:............ {total_cost:,.2f} $")
            print(f"   - Geração Total Despachada:........... {total_gen:.4f} p.u.")
            print(f"   - Carga Total Atendida (Demanda):..... {total_load_demand:.4f} p.u.")
            print(f"   - Perdas Totais Estimadas:............ {total_loss:.4f} p.u.")
            print(f"   - Balanço (Geração - Carga - Perdas):. {total_gen - total_load_demand - total_loss:,.6f} p.u.")
            print("-" * 90)

            # --- 2. DESPACHO DA GERAÇÃO E ANÁLISE DE LIMITES ---
            print("\n## 2. DESPACHO DA GERAÇÃO E ANÁLISE DE LIMITES ##")
            print(f"   {'Gerador':<12} {'Barra':<6} {'P (pu)':>10} {'Carga (%)':>11} {'P (MW)':>10} {'Custo($/h)':>12} {'λ Pmin':>10} {'λ Pmax':>10}")
            print(f"   {'-'*12:<12} {'-'*6:<6} {'-'*10:>10} {'-'*11:>11} {'-'*10:>10} {'-'*12:>12} {'-'*10:>10} {'-'*10:>10}")
            
            for g in self.net.generators:
                p_pu = g.p_var.value()
                p_mw = p_pu * self.net.sb # Supondo que a base de potência está em net.s_base
                cost = p_pu * g.cost_b
                
                # Calcula o carregamento do gerador em relação à sua capacidade máxima.
                loading = (p_pu / g.p_max * 100) if g.p_max > 0 else 0
                
                # Coeficientes de Lagrange (preços-sombra) para os limites Pmin e Pmax.
                reduced_cost = g.p_var.dj
                lambda_pmin = 0.0
                lambda_pmax = 0.0
                tol = 1e-6 # Tolerância para comparação de ponto flutuante

                if abs(p_pu - g.p_min) < tol:
                    lambda_pmin = reduced_cost
                elif abs(p_pu - g.p_max) < tol:
                    lambda_pmax = -reduced_cost
                    
                print(f"   {g.id:<12} {g.bus.id:<6} {p_pu:>10.4f} {loading:>10.2f}% {'':<1} {p_mw:>10.2f} {cost:>12.2f} {lambda_pmin:>10.2f} {lambda_pmax:>10.2f}")
            print("-" * 90)

            # --- 3. FLUXO DE POTÊNCIA NAS LINHAS ---
            print("\n## 3. FLUXO DE POTÊNCIA NAS LINHAS DE TRANSMISSÃO ##")
            print(f"   {'Linha (ID)':<12} {'De -> Para':<12} {'Fluxo (p.u.)':>15} {'Capacidade':>15} {'Carregamento (%)':>18}")
            print(f"   {'-'*12:<12} {'-'*12:<12} {'-'*15:>15} {'-'*15:>15} {'-'*18:>18}")
            for line in self.net.lines:
                flow = (line.from_bus.theta_var.value() - line.to_bus.theta_var.value()) / line.reactance
                capacity = line.flow_max_pu
                loading = (abs(flow) / capacity * 100) if capacity > 0 else 0
                print(f"   {line.id:<12} {str(line.from_bus.id)+' -> '+str(line.to_bus.id):<12} {flow:>15.4f} {capacity:>15.4f} {loading:>17.2f}%")
            print("-" * 90)

            # --- 4. CONDIÇÕES DAS BARRAS E CUSTOS MARGINAIS (LMP) ---
            print("\n## 4. CONDIÇÕES DAS BARRAS E CUSTOS MARGINAIS (LMP) ##")
            print(f"   {'Barra (ID)':<12} {'Nome':<15} {'Ângulo (graus)':>18} {'LMP ($/p.u.)':>18}")
            print(f"   {'-'*12:<12} {'-'*15:<15} {'-'*18:>18} {'-'*18:>18}")
            for b in self.net.buses:
                angle_deg = np.rad2deg(b.theta_var.value())
                lmp = 0.0
                
                if b.bus_type == "Slack":
                    if b.generators:
                        lmp = b.generators[0].cost_b
                    else:
                        lmp = 0.0
                else:
                    constraint_name = f"{b.name.replace(' ', '_')}_Power_Balance"
                    if constraint_name in self.problem.constraints:
                        lmp = -self.problem.constraints[constraint_name].pi
                    
                print(f"   {b.id:<12} {b.name:<15} {angle_deg:>18.2f} {lmp:>18.2f}")
            print("-" * 90)
            
            # --- 5. ANÁLISE DE CONGESTIONAMENTO ---
            print("\n## 5. ANÁLISE DE CONGESTIONAMENTO NA TRANSMISSÃO ##")
            # Cabeçalho da nova tabela, com colunas para os custos sombra superior e inferior
            print(f"   {'Linha (ID)':<15} {'C.S. Limite Superior ($/p.u.)':>35} {'C.S. Limite Inferior ($/p.u.)':>35}")
            print(f"   {'-'*15:<15} {'-'*35:>35} {'-'*35:>35}")

            # Itera sobre todas as linhas para buscar e exibir os custos sombra
            for line in self.net.lines:
                sanitized_name = line.name.replace(" ", "_")
                
                # Busca o preço dual (pi) da restrição de limite máximo (flow <= capacity)
                # O valor de 'pi' para uma restrição '<=' ativa é negativo. Multiplicamos por -1 para interpretá-lo como um custo positivo.
                pi_max = -self.problem.constraints[f"Flow_limit_Max_{sanitized_name}"].pi
                
                # Busca o preço dual (pi) da restrição de limite mínimo (flow >= -capacity)
                # O valor de 'pi' para uma restrição '>=' ativa já é positivo.
                pi_min = self.problem.constraints[f"Flow_limit_Inverted_{sanitized_name}"].pi
                
                # Imprime os valores para a linha atual. Um valor diferente de zero indica congestão.
                print(f"   {line.id:<15} {pi_max:>35.2f} {pi_min:>35.2f}")

            print("="*90)


        # --------------------------------------------------------------------------------------------------------------------------------------------------------------------#
   # =========================================================================
    # NEW METHODS START HERE
    # =========================================================================

    def solve_transmission(self):
        """
        Segundo estágio do PL conforme Slide 20:
        - Fixar os valores de geração PG obtidos no estágio anterior
        - Resolver novamente apenas para os ângulos (theta), como variáveis
        - Verificar que o resultado de theta é igual ao do estágio anterior
        """
        self.solve_with_loss()
        if self.problem is None or self.problem.status != pl.LpStatusOptimal:
            raise ValueError("É necessário rodar solve() ou solve_with_loss() antes de chamar solve_stage2_fix_pg()")

        print("\n=== INICIANDO SEGUNDO ESTÁGIO (Ângulos como incógnitas, PG fixos) ===")

        # Criar novo problema
        stage2 = pl.LpProblem("Linear_Economic_Dispatch_Stage2", pl.LpMinimize)

        # Variáveis apenas para ângulo
        theta_vars = {}
        for b in self.net.buses:
            if b.bus_type == "Slack":
                theta_vars[b] = pl.LpVariable(f"Angle_{b.name}_S2", lowBound=0, upBound=0)
            else:
                theta_vars[b] = pl.LpVariable(f"Angle_{b.name}_S2", lowBound=-np.pi, upBound=np.pi)
                theta_vars[b].setInitialValue(b.theta_var.value())  # usar ângulo anterior como chute

        # Objetivo nulo (ou mínimo ajuste)
        stage2 += 0, "Dummy_Objective"

        # Criar restrições de balanço com PG fixo
        for b in self.net.buses:
            generation = sum([g.p_var.value() for g in b.generators])  # PG fixo
            load = sum([l.p for l in b.loads]) + b.loss

            flow_in = 0
            flow_out = 0
            for l in self.net.lines:
                if l.from_bus == b:
                    flow_out += (theta_vars[b] - theta_vars[l.to_bus]) / l.reactance
                elif l.to_bus == b:
                    flow_in += (theta_vars[l.from_bus] - theta_vars[b]) / l.reactance

            stage2 += generation + flow_in - flow_out == load, f"{b.name}_Power_Balance_Stage2"

        # Resolver
        stage2.solve()

        print("=== RESULTADOS DO SEGUNDO ESTÁGIO ===")
        for b in self.net.buses:
            print(f"Barra {b.name} -> Theta1 = {np.rad2deg(b.theta_var.value()):.4f}°, Theta2 = {np.rad2deg(theta_vars[b].value()):.4f}°")

        return stage2

    def print_transmission_results(self):
        """
        Imprime análise de fluxo e congestionamento das linhas:
        - Fluxo real em p.u.
        - Capacidade máxima
        - Carregamento (%)
        - Custo sombra (dual) para limite superior e inferior
        """
        if self.problem is None or self.problem.status != pl.LpStatusOptimal:
            print("Modelo não resolvido ainda. Rode solve() antes.")
            return

        print("\n" + "="*90)
        print("ANÁLISE DO SISTEMA DE TRANSMISSÃO".center(90))
        print("="*90)

        print(f"   {'Linha (ID)':<15} {'De -> Para':<15} {'Fluxo (p.u.)':>15} {'Cap. (p.u.)':>12} {'Carga (%)':>12} {'CSup ($)':>12} {'CInf ($)':>12}")
        print(f"   {'-'*15:<15} {'-'*15:<15} {'-'*15:>15} {'-'*12:>12} {'-'*12:>12} {'-'*12:>12} {'-'*12:>12}")

        for line in self.net.lines:
            flow = (line.from_bus.theta_var.value() - line.to_bus.theta_var.value()) / line.reactance
            cap = line.flow_max_pu
            loading = abs(flow) / cap * 100 if cap > 0 else 0

            sanitized_name = line.name.replace(" ", "_")
            pi_max = -self.problem.constraints[f"Flow_limit_Max_{sanitized_name}"].pi
            pi_min = self.problem.constraints[f"Flow_limit_Inverted_{sanitized_name}"].pi

            print(f"   {line.id:<15} {str(line.from_bus.id)+' -> '+str(line.to_bus.id):<15} {flow:>15.4f} {cap:>12.4f} {loading:>12.2f} {pi_max:>12.2f} {pi_min:>12.2f}")

        print("="*90)


In [ ]:
class LinearDispatch:
    def __init__(self, net: Network):
        """
        Inicializa e constrói o problema de despacho econômico linear para uma dada rede.
        """
        self.net = net
        self.problem = None

        # Initializing losses on each bus:
        for b in self.net.buses:
            b.loss = 0
        
    def _create_problem(self):
        """Cria o objeto do problema PuLP."""
        self.problem = pl.LpProblem("Linear_Economic_Dispatch_OO", pl.LpMinimize)
    
    def _create_variables(self):
        """Cria as variáveis de decisão e as anexa aos objetos da rede."""
        # Geração Ativa
        for g in self.net.generators:
            g.p_var = pl.LpVariable(f"Gen_{g.name}", lowBound=g.p_min, upBound=g.p_max)

        # Ângulo das Barras
        for b in self.net.buses:
            if b.bus_type == "Slack":
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=0, upBound=0)
            else:
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=-np.pi, upBound=np.pi)
                b.theta_var.setInitialValue(0)

    def _create_objective(self):
        """Define a função objetivo do problema (minimizar custo total)."""
        self.problem += pl.lpSum([g.cost_b * g.p_var for g in self.net.generators]), "Total_Generation_Cost"

    def _create_constraints(self):
        """Cria todas as restrições do modelo."""
        # 1. Definição de Fluxo (liga as variáveis de fluxo aos ângulos)
        for line in self.net.lines:
            if line.reactance == 0:
                raise ValueError(f"Line '{line.id}' has zero reactance.")
            
            flow_eq = (line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance
            self.problem += flow_eq <= line.flow_max_pu, f"Flow_limit_Max_{line.name}"
            self.problem += flow_eq >= -line.flow_max_pu, f"Flow_limit_Inverted_{line.name}"

        for b in self.net.buses:
            generation = pl.lpSum([g.p_var for g in b.generators])
            load = sum([l.p for l in b.loads]) + b.loss
            flow_in = 0
            flow_out = 0
            for l in self.net.lines:
                if l.from_bus == b: #The line starts at bus 'b', so it's an outgoing flow
                    flow_out += (b.theta_var - l.to_bus.theta_var) / l.reactance
            
                elif l.to_bus == b: #The line ends at bus 'b', so it's an incoming flow
                    flow_in += (l.from_bus.theta_var - b.theta_var) / l.reactance

            self.problem += generation + flow_in - flow_out == load, f"{b.name}_Power_Balance"
    
    def _build(self):
        """" Monta o Problema de Otimização"""
        print("Building Optimization Problem...")
        self._create_problem()
        self._create_variables()
        self._create_constraints()
        self._create_objective()

    def solve(self):
        """Resolve o problema de otimização."""
        self._build()
        print("Solving...")
        self.problem.solve()
        return self.problem.status == pl.LpStatusOptimal

    
    def solve_with_loss(self, iter_max=10, tol=1e-10):
        if not self.solve():
            raise ValueError("Não foi possível resolver o despacho inicial sem perdas")
        
        prev_total_loss = 0
        for i in range(iter_max):
            print(f"\n--- Iniciando Iteração {i+1} de Cálculo de Perdas ---")

            for b in self.net.buses:
                b.loss = 0
            
            current_total_loss = 0
            for l in self.net.lines:
                r = l.resistance
                x = l.reactance
                g_series = r / (r**2 + x**2) if (r**2 + x**2) > 0 else 0
                dtheta = l.from_bus.theta_var.value() - l.to_bus.theta_var.value()
                line_loss = g_series * (dtheta ** 2)
                current_total_loss += line_loss
                l.from_bus.loss += line_loss / 2
                l.to_bus.loss += line_loss / 2
            
            #Verificação de Convergência
            loss_diff = abs(current_total_loss - prev_total_loss)
            print(f"Mudança nas perdas desde a última iteração: {loss_diff:.6f} pu")
            if loss_diff < tol:
                print(f"Convergência atingida. A mudança nas perdas está abaixo da tolerância")
                break
            prev_total_loss = current_total_loss

            if not self.solve():
                print(f"Aviso: Não foi possível resolver o despacho com perdas. Erro nas iteração {i}")
            
        print("\n Processo de despacho com perdas finalizado")
    
    def solve_transmission(self):
        #Etapa 1: Rodar o PL com perdas
        self.solve_with_loss()

        #Etapa 2: Armazenar as gerações ótimas
        for g in self.net.generators:
            g.p_opt = g.p_var.value()
        
        #Etapa 3: Determinar o sentido do fluxo de cada linha
        for line in self.net.lines:
            flow_value = (line.from_bus.theta_var.value() - line.to_bus.theta_var.value()) / line.reactance
            line.flow_sign = 1 if flow_value >= 0 else -1


        #Etapa 4: Montar o problema
        self._create_problem()
        
        #Etapa 5: Criar as variáveis
        # Ângulo das Barras
        for b in self.net.buses:
            if b.bus_type == "Slack":
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=0, upBound=0)
            else:
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=-np.pi, upBound=np.pi)
                b.theta_var.setInitialValue(0)
        
        #Etapa 6: Criar as restrições

        # 1. Definição de Fluxo (liga as variáveis de fluxo aos ângulos)
        for line in self.net.lines:
            if line.reactance == 0:
                raise ValueError(f"Line '{line.id}' has zero reactance.")
            
            line.flow_eq = (line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance
            self.problem += line.flow_eq <= line.flow_max_pu, f"Flow_limit_Max_{line.name}"
            self.problem += line.flow_eq >= -line.flow_max_pu, f"Flow_limit_Inverted_{line.name}"

        #2. Restrição de Balanço Nodal
        for b in self.net.buses:
            generation = sum([g.p_opt for g in b.generators])
            load = sum([l.p for l in b.loads]) + b.loss
            flow_in = 0
            flow_out = 0
            for l in self.net.lines:
                if l.from_bus == b: #The line starts at bus 'b', so it's an outgoing flow
                    flow_out += (b.theta_var - l.to_bus.theta_var) / l.reactance
            
                elif l.to_bus == b: #The line ends at bus 'b', so it's an incoming flow
                    flow_in += (l.from_bus.theta_var - b.theta_var) / l.reactance

            self.problem += flow_in - flow_out == load - generation, f"{b.name}_Power_Balance"       


        #Etapa 7: Criar a FOB
        self.problem += pl.lpSum([line.flow_max_pu * line.flow_sign * line.flow_eq for line in self.net.lines]), "Total_Flow_Cost"

        #Etapa 8: Resolver o problema
        print("--> Etapa 7: Resolvendo o problema de sinais de expansão...")
        self.problem.solve()
        if self.problem.status != pl.LpStatusOptimal:
            print("\nERRO: Não foi possível resolver o problema para obter os sinais de expansão.")
            return False
        return True


    def print_model(self):
        print(self.problem)

    def print_results(self):
            """
            Imprime um relatório completo e organizado dos resultados da última solução,
            seguindo um formato tabular e estruturado.
            """
            if self.problem is None or self.problem.status != pl.LpStatusOptimal:
                status = "Não resolvido" if self.problem is None else pl.LpStatus[self.problem.status]
                print(f"Não foi possível encontrar uma solução ótima. Status: {status}")
                return

            print("\n" + "="*90)
            print("RELATÓRIO DO DESPACHO ECONÔMICO".center(90))
            print("="*90)

            # --- 1. RESUMO GERAL DO SISTEMA ---
            total_gen = sum(g.p_var.value() for g in self.net.generators)
            total_load_demand = sum(l.p for l in self.net.loads)
            total_loss = sum(b.loss for b in self.net.buses)
            total_cost = pl.value(self.problem.objective)

            print("\n## 1. RESUMO GERAL DO SISTEMA ##")
            print(f"   - Custo Total de Operação:............ {total_cost:,.2f} $")
            print(f"   - Geração Total Despachada:........... {total_gen:.4f} p.u.")
            print(f"   - Carga Total Atendida (Demanda):..... {total_load_demand:.4f} p.u.")
            print(f"   - Perdas Totais Estimadas:............ {total_loss:.4f} p.u.")
            print(f"   - Balanço (Geração - Carga - Perdas):. {total_gen - total_load_demand - total_loss:,.6f} p.u.")
            print("-" * 90)

            # --- 2. DESPACHO DA GERAÇÃO E ANÁLISE DE LIMITES ---
            print("\n## 2. DESPACHO DA GERAÇÃO E ANÁLISE DE LIMITES ##")
            print(f"   {'Gerador':<12} {'Barra':<6} {'P (pu)':>10} {'Carga (%)':>11} {'P (MW)':>10} {'Custo($/h)':>12} {'λ Pmin':>10} {'λ Pmax':>10}")
            print(f"   {'-'*12:<12} {'-'*6:<6} {'-'*10:>10} {'-'*11:>11} {'-'*10:>10} {'-'*12:>12} {'-'*10:>10} {'-'*10:>10}")
            
            for g in self.net.generators:
                p_pu = g.p_var.value()
                p_mw = p_pu * self.net.sb # Supondo que a base de potência está em net.s_base
                cost = p_pu * g.cost_b
                
                # Calcula o carregamento do gerador em relação à sua capacidade máxima.
                loading = (p_pu / g.p_max * 100) if g.p_max > 0 else 0
                
                # Coeficientes de Lagrange (preços-sombra) para os limites Pmin e Pmax.
                reduced_cost = g.p_var.dj
                lambda_pmin = 0.0
                lambda_pmax = 0.0
                tol = 1e-6 # Tolerância para comparação de ponto flutuante

                if abs(p_pu - g.p_min) < tol:
                    lambda_pmin = reduced_cost
                elif abs(p_pu - g.p_max) < tol:
                    lambda_pmax = -reduced_cost
                    
                print(f"   {g.id:<12} {g.bus.id:<6} {p_pu:>10.4f} {loading:>10.2f}% {'':<1} {p_mw:>10.2f} {cost:>12.2f} {lambda_pmin:>10.2f} {lambda_pmax:>10.2f}")
            print("-" * 90)

            # --- 3. FLUXO DE POTÊNCIA NAS LINHAS ---
            print("\n## 3. FLUXO DE POTÊNCIA NAS LINHAS DE TRANSMISSÃO ##")
            print(f"   {'Linha (ID)':<12} {'De -> Para':<12} {'Fluxo (p.u.)':>15} {'Capacidade':>15} {'Carregamento (%)':>18}")
            print(f"   {'-'*12:<12} {'-'*12:<12} {'-'*15:>15} {'-'*15:>15} {'-'*18:>18}")
            for line in self.net.lines:
                flow = (line.from_bus.theta_var.value() - line.to_bus.theta_var.value()) / line.reactance
                capacity = line.flow_max_pu
                loading = (abs(flow) / capacity * 100) if capacity > 0 else 0
                print(f"   {line.id:<12} {str(line.from_bus.id)+' -> '+str(line.to_bus.id):<12} {flow:>15.4f} {capacity:>15.4f} {loading:>17.2f}%")
            print("-" * 90)

            # --- 4. CONDIÇÕES DAS BARRAS E CUSTOS MARGINAIS (LMP) ---
            print("\n## 4. CONDIÇÕES DAS BARRAS E CUSTOS MARGINAIS (LMP) ##")
            print(f"   {'Barra (ID)':<12} {'Nome':<15} {'Ângulo (graus)':>18} {'LMP ($/p.u.)':>18}")
            print(f"   {'-'*12:<12} {'-'*15:<15} {'-'*18:>18} {'-'*18:>18}")
            for b in self.net.buses:
                angle_deg = np.rad2deg(b.theta_var.value())
                lmp = 0.0
                
                if b.bus_type == "Slack":
                    if b.generators:
                        lmp = b.generators[0].cost_b
                    else:
                        lmp = 0.0
                else:
                    constraint_name = f"{b.name.replace(' ', '_')}_Power_Balance"
                    if constraint_name in self.problem.constraints:
                        lmp = -self.problem.constraints[constraint_name].pi
                    
                print(f"   {b.id:<12} {b.name:<15} {angle_deg:>18.2f} {lmp:>18.2f}")
            print("-" * 90)
            
            # --- 5. ANÁLISE DE CONGESTIONAMENTO ---
            print("\n## 5. ANÁLISE DE CONGESTIONAMENTO NA TRANSMISSÃO ##")
            # Cabeçalho da nova tabela, com colunas para os custos sombra superior e inferior
            print(f"   {'Linha (ID)':<15} {'C.S. Limite Superior ($/p.u.)':>35} {'C.S. Limite Inferior ($/p.u.)':>35}")
            print(f"   {'-'*15:<15} {'-'*35:>35} {'-'*35:>35}")

            # Itera sobre todas as linhas para buscar e exibir os custos sombra
            for line in self.net.lines:
                sanitized_name = line.name.replace(" ", "_")
                
                # Busca o preço dual (pi) da restrição de limite máximo (flow <= capacity)
                # O valor de 'pi' para uma restrição '<=' ativa é negativo. Multiplicamos por -1 para interpretá-lo como um custo positivo.
                pi_max = -self.problem.constraints[f"Flow_limit_Max_{sanitized_name}"].pi
                
                # Busca o preço dual (pi) da restrição de limite mínimo (flow >= -capacity)
                # O valor de 'pi' para uma restrição '>=' ativa já é positivo.
                pi_min = self.problem.constraints[f"Flow_limit_Inverted_{sanitized_name}"].pi
                
                # Imprime os valores para a linha atual. Um valor diferente de zero indica congestão.
                print(f"   {line.id:<15} {pi_max:>35.2f} {pi_min:>35.2f}")

            print("="*90)

    def _generate_detailed_report(self):
        """Imprime um relatório detalhado e completo da última solução do problema."""
        if self.problem is None or self.problem.status != pl.LpStatusOptimal:
            print("Problema não resolvido ou sem solução ótima. Não é possível gerar o relatório.")
            return

        print("\n" + "="*80)
        print("RELATÓRIO DETALHADO DA ANÁLISE DE TRANSMISSÃO".center(80))
        print("="*80)

        # --- SEÇÃO 0: RESUMO GERAL ---
        fob_value = pl.value(self.problem.objective)
        total_gen = sum(g.p_opt for g in self.net.generators)
        total_load = sum(l.p for l in self.net.loads)
        total_loss = total_gen - total_load # Perdas são a diferença
        
        print("\n## 0. RESUMO GERAL DO SISTEMA ##")
        print(f"  - Valor da FOB (Custo de Fluxo Ponderado): {fob_value:,.2f} $")
        print(f"  - Geração Total (fixada):................. {total_gen:.4f} p.u.")
        print(f"  - Carga Total Atendida:................... {total_load:.4f} p.u.")
        print(f"  - Perdas Totais do Sistema:............... {total_loss:.4f} p.u.")
        
        print("-" * 80)

        # --- SEÇÃO 1: GERAÇÃO POR UNIDADE ---
        print("\n## 1. DESPACHO DA GERAÇÃO (Valores Fixados) ##")
        print(f"{'Gerador (ID)':<15} {'Barra':<10} {'Geração (p.u.)':<15}")
        for g in self.net.generators:
            print(f"{g.id:<15} {g.bus.id:<10} {g.p_opt:<15.4f}")

        print("-" * 80)

        # --- SEÇÃO 2: FLUXO NAS LINHAS ---
        print("\n## 2. FLUXO DE POTÊNCIA NAS LINHAS DE TRANSMISSÃO ##")
        print(f"{'Linha (ID)':<12} {'De -> Para':<12} {'Fluxo (p.u.)':>15} {'Capacidade':>15} {'Carregamento (%)':>18}")
        for line in self.net.lines:
            flow = line.flow_eq.value()
            capacity = line.flow_max_pu
            loading = (abs(flow) / capacity * 100) if capacity > 0 else 0
            print(f"{line.id:<12} {str(line.from_bus.id)+' -> '+str(line.to_bus.id):<12} {flow:>15.4f} {capacity:>15.4f} {loading:>17.2f}%")

        print("-" * 80)
        
        # --- SEÇÃO 3: ANÁLISE DAS RESTRIÇÕES (COEFICIENTES DE LAGRANGE) ---
        print("\n## 3. ANÁLISE ECONÔMICA DAS RESTRIÇÕES (PREÇOS-SOMBRA) ##")
        
        # 3.1 Custo Marginal de Operação (preço-sombra do balanço de potência)
        print("\n  >> Custo Marginal de Operação (CMO) por Barra")
        print(f"  {'Barra (ID)':<12} {'CMO ($/p.u.)':<15}")
        for b in self.net.buses:
            if b.bus_type != "Slack":
                constraint_name = f"{b.name.replace(' ', '_')}_Power_Balance"
                cmo = -self.problem.constraints[constraint_name].pi
                print(f"  {b.id:<12} {cmo:<15.2f}")

        # 3.2 Sinais de Congestionamento (preço-sombra dos limites de fluxo)
        print("\n  >> Sinais de Congestionamento por Linha (separados por limite)")
        print(f"  {'Linha (ID)':<12} {'Limite Superior (pi)':<25} {'Limite Inferior (pi)':<25}")
        for line in self.net.lines:
            c_max_name = f"Flow_limit_Max_{line.name.replace(' ', '_')}"
            c_min_name = f"Flow_limit_Inverted_{line.name.replace(' ', '_')}"
            
            pi_max = self.problem.constraints[c_max_name].pi
            pi_min = self.problem.constraints[c_min_name].pi
            
            # Imprime apenas se um dos limites tiver um preço-sombra relevante
            if abs(pi_max) > 1e-6 or abs(pi_min) > 1e-6:
                print(f"  {line.id:<12} {pi_max:<25.2f} {pi_min:<25.2f}")
        
        print("\n" + "="*80)

    def generate_detailed_report(self):
            """
            Imprime um relatório completo e organizado dos resultados da análise de sinais de expansão,
            seguindo um formato tabular e estruturado de 5 seções.
            Este método deve ser chamado APÓS a execução de 'solve_transmission'.
            """
            if self.problem is None or self.problem.status != pl.LpStatusOptimal:
                status = "Não resolvido" if self.problem is None else pl.LpStatus[self.problem.status]
                print(f"Não foi possível encontrar uma solução ótima para a análise de transmissão. Status: {status}")
                return

            print("\n" + "="*100)
            print("RELATÓRIO DA ANÁLISE DE SINAIS DE EXPANSÃO DA TRANSMISSÃO".center(100))
            print("="*100)

            # --- 1. RESUMO GERAL DO SISTEMA ---
            fob_value = pl.value(self.problem.objective)
            total_gen = sum(g.p_opt for g in self.net.generators)
            total_load_demand = sum(l.p for l in self.net.loads)
            total_loss = sum(b.loss for b in self.net.buses)

            print("\n## 1. RESUMO GERAL DO SISTEMA ##")
            print(f"   - Valor da FOB (Sinal de Expansão):....... {fob_value:,.2f} $")
            print(f"   - Geração Total (Fixada):................. {total_gen:.4f} p.u.")
            print(f"   - Carga Total Atendida (Demanda):......... {total_load_demand:.4f} p.u.")
            print(f"   - Perdas Totais Estimadas:................ {total_loss:.4f} p.u.")
            print(f"   - Balanço (Geração - Carga - Perdas):..... {total_gen - total_load_demand - total_loss:,.6f} p.u.")
            print("-" * 100)

            # --- 2. GERAÇÃO FIXADA (INPUT DO PROBLEMA) ---
            print("\n## 2. GERAÇÃO FIXADA PARA A ANÁLISE ##")
            print(f"   {'Gerador':<15} {'Barra':<10} {'P (pu)':>15}")
            print(f"   {'-'*15:<15} {'-'*10:<10} {'-'*15:>15}")
            for g in self.net.generators:
                print(f"   {g.id:<15} {g.bus.id:<10} {g.p_opt:>15.4f}")
            print("-" * 100)

            # --- 3. FLUXO DE POTÊNCIA NAS LINHAS ---
            print("\n## 3. FLUXO DE POTÊNCIA NAS LINHAS DE TRANSMISSÃO ##")
            print(f"   {'Linha (ID)':<12} {'De -> Para':<12} {'Fluxo (p.u.)':>15} {'Capacidade':>15} {'Carregamento (%)':>20}")
            print(f"   {'-'*12:<12} {'-'*12:<12} {'-'*15:>15} {'-'*15:>15} {'-'*20:>20}")
            for line in self.net.lines:
                # A expressão do fluxo já foi criada e está em 'line.flow_eq'
                flow = line.flow_eq.value()
                capacity = line.flow_max_pu
                loading = (abs(flow) / capacity * 100) if capacity > 0 else 0
                print(f"   {line.id:<12} {str(line.from_bus.id)+' -> '+str(line.to_bus.id):<12} {flow:>15.4f} {capacity:>15.4f} {loading:>19.2f}%")
            print("-" * 100)

            # --- 4. CONDIÇÕES DAS BARRAS E PREÇOS-SOMBRA DE ENERGIA ---
            print("\n## 4. CONDIÇÕES DAS BARRAS E PREÇOS-SOMBRA (CMO) ##")
            print(f"   {'Barra (ID)':<12} {'Nome':<15} {'Ângulo (graus)':>20} {'CMO ($/p.u.)':>20}")
            print(f"   {'-'*12:<12} {'-'*15:<15} {'-'*20:>20} {'-'*20:>20}")
            for b in self.net.buses:
                angle_deg = np.rad2deg(b.theta_var.value())
                cmo = 0.0 # Custo Marginal de Operação (ou preço-sombra da energia)
                
                # O preço-sombra vem da restrição de balanço de potência
                constraint_name = f"{b.name.replace(' ', '_')}_Power_Balance"
                if constraint_name in self.problem.constraints:
                    cmo = -self.problem.constraints[constraint_name].pi
                else:
                    # Para a barra Slack, não há restrição explícita, o preço é a referência (0 neste contexto)
                    cmo = 0.0
                    
                print(f"   {b.id:<12} {b.name:<15} {angle_deg:>20.2f} {cmo:>20.2f}")
            print("-" * 100)
            
            # --- 5. ANÁLISE DE CONGESTIONAMENTO ---
            print("\n## 5. ANÁLISE DE CONGESTIONAMENTO NA TRANSMISSÃO ##")
            # Cabeçalho da tabela, com colunas para os custos sombra superior e inferior
            print(f"   {'Linha (ID)':<15} {'C.S. Limite Superior ($/p.u.)':>35} {'C.S. Limite Inferior ($/p.u.)':>35}")
            print(f"   {'-'*15:<15} {'-'*35:>35} {'-'*35:>35}")

            # Itera sobre todas as linhas para buscar e exibir os custos sombra
            for line in self.net.lines:
                sanitized_name = line.name.replace(" ", "_")
                
                # Busca o preço dual (pi) da restrição de limite máximo (flow <= capacity)
                # O valor de 'pi' para uma restrição '<=' ativa é negativo. Multiplicamos por -1 para interpretá-lo como um custo positivo.
                pi_max = -self.problem.constraints[f"Flow_limit_Max_{sanitized_name}"].pi
                
                # Busca o preço dual (pi) da restrição de limite mínimo (flow >= -capacity)
                # O valor de 'pi' para uma restrição '>=' ativa já é positivo.
                pi_min = self.problem.constraints[f"Flow_limit_Inverted_{sanitized_name}"].pi
                
                # Imprime os valores para a linha atual. Um valor diferente de zero indica congestão.
                if pi_max > 1e-6 or pi_min > 1e-6:
                    print(f"   {line.id:<15} {pi_max:>35.8f} {pi_min:>35.8f}")

            print("="*100)

    
    def solve_transmission_2(self):
        #Etapa 1: Rodar o PL com perdas
        self.solve_with_loss()

        #Etapa 2: Armazenar as gerações ótimas
        for g in self.net.generators:
            g.p_opt = g.p_var.value()
        
        #Etapa 3: Determinar o sentido do fluxo de cada linha
        for line in self.net.lines:
            flow_value = (line.from_bus.theta_var.value() - line.to_bus.theta_var.value()) / line.reactance
            line.flow_sign = 1 if flow_value >= 0 else -1


        #Etapa 4: Montar o problema
        self._create_problem()
        
        #Etapa 5: Criar as variáveis
        # Ângulo das Barras
        for b in self.net.buses:
            if b.bus_type == "Slack":
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=0, upBound=0)
            else:
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=-np.pi, upBound=np.pi)
                b.theta_var.setInitialValue(0)
        
        # Fluxo nas linhas
        for l in self.net.lines:
            l.flux_var = pl.LpVariable(f"Fluxo na {l.name}")
            l.cost = l.flow_max_pu
        
        #Etapa 6: Criar as restrições

        # 1. Definição de Fluxo (liga as variáveis de fluxo aos ângulos)
        for line in self.net.lines:
            if line.reactance == 0:
                raise ValueError(f"Line '{line.id}' has zero reactance.")
            
            flow_from_angles = (line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance
            self.problem += line.flux_var == flow_from_angles, f"Flow_Def_{line.name}" 

        #2. Restrição de Balanço Nodal
        for b in self.net.buses:
            generation = sum([g.p_opt for g in b.generators])
            load = sum([l.p for l in b.loads]) + b.loss
            flow_in = 0
            flow_out = 0
            for l in self.net.lines:
                if l.from_bus == b: #The line starts at bus 'b', so it's an outgoing flow
                    flow_out += (b.theta_var - l.to_bus.theta_var) / l.reactance
            
                elif l.to_bus == b: #The line ends at bus 'b', so it's an incoming flow
                    flow_in += (l.from_bus.theta_var - b.theta_var) / l.reactance

            self.problem += flow_in - flow_out == load - generation, f"{b.name}_Power_Balance"       


        #Etapa 7: Criar a FOB
        self.problem += pl.lpSum([line.flow_max_pu * line.flow_sign * line.flow_eq for line in self.net.lines]), "Total_Flow_Cost"

        #Etapa 8: Resolver o problema
        print("--> Etapa 7: Resolvendo o problema de sinais de expansão...")
        self.problem.solve()
        if self.problem.status != pl.LpStatusOptimal:
            print("\nERRO: Não foi possível resolver o problema para obter os sinais de expansão.")
            return False
        return True


# Resultados com Perdas

In [21]:
b6 = B6L8()
solver = LinearDispatch(b6)
solver.solve_with_loss()
solver.print_results()

Building Optimization Problem...
Solving...

--- Iniciando Iteração 1 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.003874 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 2 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000025 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 3 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 4 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 5 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Convergência atingida. A mudança nas perdas está abaixo da tolerância

 Processo de despacho com perdas finalizado

                             RELATÓRIO DO DESPACHO ECONÔMICO                              

## 1. RESUMO GERAL DO SISTEMA ##
   -

In [22]:
ieee118 = IEEE118()
solver = LinearDispatch(ieee118)
solver.solve_with_loss()
solver.print_results()

Building Optimization Problem...
Solving...

--- Iniciando Iteração 1 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.776661 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 2 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.013873 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 3 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000579 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 4 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000020 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 5 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000001 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 6 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 7 de Cálculo de Perdas ---

# Resultados Investimento em LT

In [19]:
ieee118 = IEEE118()
solver = LinearDispatch2(ieee118)
solver.solve_transmission()
solver.print_transmission_results()

Building Optimization Problem...
Solving...

--- Iniciando Iteração 1 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.776661 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 2 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.013873 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 3 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000579 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 4 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000020 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 5 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000001 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 6 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 7 de Cálculo de Perdas ---

TypeError: unhashable type: 'Bus'

In [12]:
b6 = B6L8()
solver = LinearDispatch2(b6)
solver.solve_transmission()
solver.print_transmission_results()



          TRANSMISSION INVESTMENT ANALYSIS: STEP 1 - ECONOMIC DISPATCH          
Building Optimization Problem...
Solving...
Despacho econômico inicial resolvido. PGs ótimos obtidos.
--------------------------------------------------------------------------------

        TRANSMISSION INVESTMENT ANALYSIS: STEP 2 - MINIMUM INVESTMENT LP        
Resolvendo o problema de investimento mínimo...
Solução ótima de investimento encontrada.

                         RELATÓRIO DO INVESTIMENTO EM TRANSMISSÃO                         

## 1. RESUMO DO INVESTIMENTO ##
 - Custo Total do Investimento: 0.00 $
------------------------------------------------------------------------------------------

## 2. CAPACIDADE ADICIONADA POR LINHA (INVESTIMENTO ÓTIMO) ##
   Linha (ID)           Investimento (Δf em p.u.)     
   -------------------- ------------------------------
------------------------------------------------------------------------------------------

## 3. VARIÁVEIS DUAIS (PREÇOS SOMBRA) DAS R

In [37]:
ieee118 = IEEE118()
solver = LinearDispatch(ieee118)
solver.solve_transmission()
solver.generate_detailed_report()

Building Optimization Problem...
Solving...

--- Iniciando Iteração 1 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.776661 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 2 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.013873 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 3 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000579 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 4 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000020 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 5 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000001 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 6 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 7 de Cálculo de Perdas ---

# LT novo metodo

In [42]:
ieee118 = IEEE118()
solver = LinearDispatch(ieee118)
solver.solve_transmission_flow_model()
solver.generate_signed_flow_model_report()

--> Etapa 1: Resolvendo o Despacho Econômico com Perdas...
Building Optimization Problem...
Solving...

--- Iniciando Iteração 1 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.776661 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 2 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.013873 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 3 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000579 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 4 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000020 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 5 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000001 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 6 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
So